# day 14

https://adventofcode.com/2019/day/14

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day14.txt')

LOGGER = logging.getLogger('day14')

## part 1

### problem statement:

#### loading data

In [ ]:
test_0 = """9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL"""

test_1 = """157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT"""

test_2 = """2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF"""

test_3 = """171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
import re

from collections import defaultdict
from copy import deepcopy


def parse_reactions(s):
    equivalencies = {}
    times_needed = defaultdict(int)
    for line in s.split('\n'):
        LOGGER.debug(line)
        inp, out = line.strip().split(' => ')
        inp = inp.split(', ')
        
        out_num, out_chem = re.match('(\d+) (\w+)', out).groups()
        out_num = int(out_num)
        
        LOGGER.debug(inp)
        inps = [re.match('(\d+) (\w+)', _).groups() for _ in inp]

        inp_dict = {inp_chem: int(inp_num)
                    for (inp_num, inp_chem) in inps}
        equivalencies[out_chem] = (out_num, inp_dict)

        # track how often different chemicals are needed
        for inp_chem in inp_dict:
            times_needed[inp_chem] += 1
            
    return equivalencies, times_needed

In [ ]:
equivalencies, times_needed = parse_reactions(test_0)
equivalencies

In [ ]:
times_needed

#### function def

In [ ]:
ORE = 'ORE'
FUEL = 'FUEL'

def get_fuel(n_fuel, equivalencies, times_needed):
    times_needed = deepcopy(times_needed)
    times_needed[FUEL] = 0
    required_chems = {FUEL: n_fuel}
    while True:
        for chem in times_needed:
            if times_needed[chem] == 0:
                LOGGER.debug(f'we can now process request for {chem}')
                n_required = required_chems[chem]
                LOGGER.debug(f'we need {n_required} units of {chem}')
                if chem == 'ORE':
                    return n_required
                (n_output_per_reaction, reaction_inputs) = equivalencies[chem]
                n_reactions = (n_required + n_output_per_reaction - 1) // n_output_per_reaction
                LOGGER.debug(f'we will execute this reaction {n_reactions} times')
                for (needed_chem, n_needed_chem) in reaction_inputs.items():
                    if needed_chem not in required_chems:
                        required_chems[needed_chem] = 0
                    required_chems[needed_chem] += n_reactions * n_needed_chem
                    times_needed[needed_chem] -= 1
                del times_needed[chem]
                break

In [ ]:
get_fuel(1, equivalencies, times_needed)

In [ ]:
def q_1(data):
    equivalencies, times_needed = parse_reactions(data)
    return get_fuel(1, equivalencies, times_needed)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_0) == 165
    assert q_1(test_1) == 13_312
    assert q_1(test_2) == 180_697
    assert q_1(test_3) == 2_210_736
    LOGGER.setLevel(logging.INFO)

In [ ]:
print(test_1)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
import math

def q_2(data):
    equivalencies, times_needed = parse_reactions(data)
    # binary search
    left = 0
    right = int(1e12)  # 1,000,000,000,000
    while left < right:
        mid = math.ceil((left + right) / 2)
        LOGGER.info(f"mid = {mid:,}")
        c = get_fuel(mid, equivalencies, times_needed)
        LOGGER.debug(f"get_fuel({mid}, ...) = {c}")
        if c <= int(1e12):
            left = mid
        else:
            right = mid - 1
    return left

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_1) == 82_892_753
    assert q_2(test_2) == 5_586_022
    assert q_2(test_3) == 460_664
    LOGGER.setLevel(logging.INFO)

In [ ]:
q_2(test_1)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin